In [16]:
import joblib
import pickle
import warnings
%matplotlib inline
import datetime,time
import numpy as np
import pandas as pd
import seaborn as sns
import tensorflow as tf
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
warnings.filterwarnings("ignore")
plt.style.use("seaborn-whitegrid")
sns.set(rc={'figure.figsize':(12,3)})
pd.set_option('display.max_columns', 50)
import plotly.express as px
import plotly.graph_objects as go
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Dropout
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression
from xgboost import XGBRegressor,XGBClassifier
from lightgbm import LGBMRegressor,LGBMClassifier
from catboost import CatBoostRegressor,CatBoostClassifier
from sklearn.base import BaseEstimator, TransformerMixin
from tqdm.notebook import tqdm_notebook
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import SplineTransformer
from sklearn.compose import ColumnTransformer, make_column_selector
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.ensemble import RandomForestRegressor,RandomForestClassifier
from sklearn.metrics import mean_squared_error,r2_score,mean_absolute_error
from sklearn.preprocessing import StandardScaler,LabelEncoder,PolynomialFeatures
from sklearn.model_selection import train_test_split, KFold,cross_val_score,cross_validate,GridSearchCV

In [17]:
le=LabelEncoder()
scaler = StandardScaler()
train = pd.read_csv('air/Train.csv')
test = pd.read_csv('air/Test.csv')
train['ind'] = 1
test['ind'] = 0
df= pd.concat([train, test])
obj=df.select_dtypes('object')
num=df.select_dtypes(exclude='object')
col=df.drop(['id','pm2_5',],axis=1).columns
df

,id,site_id,site_latitude,site_longitude,city,country,date,hour,sulphurdioxide_so2_column_number_density,sulphurdioxide_so2_column_number_density_amf,sulphurdioxide_so2_slant_column_number_density,sulphurdioxide_cloud_fraction,sulphurdioxide_sensor_azimuth_angle,sulphurdioxide_sensor_zenith_angle,sulphurdioxide_solar_azimuth_angle,sulphurdioxide_solar_zenith_angle,sulphurdioxide_so2_column_number_density_15km,month,carbonmonoxide_co_column_number_density,carbonmonoxide_h2o_column_number_density,carbonmonoxide_cloud_height,carbonmonoxide_sensor_altitude,carbonmonoxide_sensor_azimuth_angle,carbonmonoxide_sensor_zenith_angle,carbonmonoxide_solar_azimuth_angle,...,ozone_cloud_fraction,ozone_sensor_azimuth_angle,ozone_sensor_zenith_angle,ozone_solar_azimuth_angle,ozone_solar_zenith_angle,uvaerosollayerheight_aerosol_height,uvaerosollayerheight_aerosol_pressure,uvaerosollayerheight_aerosol_optical_depth,uvaerosollayerheight_sensor_zenith_angle,uvaerosollayerheight_sensor_azimuth_angle,uvaerosollayerheight_solar_azimuth_angle,uvaerosollayerheight_solar_zenith_angle,cloud_cloud_fraction,cloud_cloud_top_pressure,cloud_cloud_top_height,cloud_cloud_base_pressure,cloud_cloud_base_height,cloud_cloud_optical_depth,cloud_surface_albedo,cloud_sensor_azimuth_angle,cloud_sensor_zenith_angle,cloud_solar_azimuth_angle,cloud_solar_zenith_angle,pm2_5,ind
0,id_vjcx08sz91,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-10-25,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.906039,-100.805145,21.720518,-123.523796,33.745914,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.015000,1
1,id_bkg215syli,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-02,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,0.045475,3771.027210,3399.756845,828569.623806,69.245351,59.159695,-143.370575,...,0.364713,70.875954,59.220097,-143.480164,26.525513,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,42.267200,1
2,id_oui2pot3qd,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-03,13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.754163,-96.411942,61.045010,-121.307121,41.898113,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.756392,45185.499590,6791.682888,51171.802486,5791.682829,11.816715,0.192757,-96.411890,61.045123,-121.307414,41.898269,39.450741,1
3,id_9aandqzy4n,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-08,14,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.000000,-95.863083,65.508858,-122.218231,43.923038,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10.537600,1
4,id_ali5x2m4iw,6531a46a89b3300013914a36,6.53257,3.39936,Lagos,Nigeria,2023-11-09,13,0.000267,0.774656,0.000207,0.223403,-97.811241,49.513344,-126.064468,40.167336,0.000083,11.0,0.049045,3514.042054,1678.370478,828902.790450,-96.099639,49.432796,-126.070747,...,0.226150,-97.811241,49.513344,-126.064468,40.167336,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.226141,86197.531250,1451.050659,96215.906250,451.050598,10.521009,0.153114,-97.811241,49.513439,-126.064453,40.167355,19.431731,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2778,id_5yblexf7zp,64654b4dd5320e001d6c499a,-0.10098,34.76242,Kisumu,Kenya,2024-01-27,10,0.000099,0.660385,0.000066,0.209272,72.440125,55.280411,-150.586182,21.271597,0.000042,1.0,0.031435,1632.436685,1732.968059,829347.270009,70.736862,55.550926,-150.697998,...,0.551999,72.410332,55.522726,-150.770801,21.260554,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.206510,59925.188475,4975.111702,67328.459370,3975.111746,2.831732,0.250637,72.428178,55.404121,-150.645025,21.248325,NaN,0
2779,id_c9cycuvetl,65c8c557b3d86f0012b2e32b,3.89696,11.50603,Yaoundé,Cameroon,2024-02-21,12,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.843743,-101.385143,16.5720

In [18]:
def drop_high_missing_columns(df, missing_threshold=0.6):
    missing_percentages = df.isnull().mean(axis=0)
    columns_to_drop = missing_percentages[missing_percentages >= missing_threshold].index
    return df.drop(columns_to_drop, axis=1)
df = drop_high_missing_columns(df)  
df.shape

(10854, 74)

In [19]:
df['city'] = le.fit_transform(df['city'])
df['country'] = le.fit_transform(df['country'])
df['date']=pd.to_datetime(df['date'])
df['year'] = pd.to_datetime(df['date']).dt.year
df['month'] = pd.to_datetime(df['date']).dt.month
df['weekofyear'] = pd.to_datetime(df['date']).dt.weekofyear
df['day'] = pd.to_datetime(df['date']).dt.day
df['sin_week_year'] = np.sin(2 * np.pi * df['weekofyear'] / 52)
df['sin_month'] = np.sin(2 * np.pi * df['month'] / 12)
df['sin_day'] = np.sin(2 * np.pi * df['day'] / 30)
df['cos_week_year'] = np.cos(2 * np.pi * df['weekofyear'] / 52)
df['cos_month'] = np.cos(2 * np.pi * df['month'] / 12)
df['cos_day'] = np.cos(2 * np.pi * df['day'] / 30)
df['extreme_low'] = (df['year'] == 2023) & ((df['month'] ==6) |(df['month'] == 8) |(df['month'] == 11))
df['low_month'] = (df['month'] ==1) |(df['month'] == 5) |(df['month'] == 6) |(df['month'] == 8) |(df['month'] == 10)  |(df['month'] == 11)
df.fillna(df.mean(),inplace=True)

In [20]:
train = df[df.ind == 1]
test = df[df.ind == 0]
col=df.drop(['id','pm2_5','site_id','date'],axis=1).columns
X=train[col]
test_df=test[col]
X=scaler.fit_transform(X)
test_df = scaler.fit_transform(test_df)
y=train.pm2_5
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
X.shape

(8071, 81)

In [22]:
# Define the model
model = Sequential()
# Input layer with 97 input dimensions and first hidden layer with 128 neurons
model.add(Dense(128, input_dim=81, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1))
# Compile the model
model.compile(optimizer='adam', loss='mean_squared_error')
# Print the model summary
model.summary()
# Train the model
model.fit(X_train, y_train, epochs=50, batch_size=32, validation_split=0.2)
# Evaluate the model on the test set
loss = model.evaluate(X_test, y_test)
print(f'Test loss: {loss}')
# Example prediction
y_pred = model.predict(X_test)
mean_squared_error(y_test, y_pred, squared=False)

Model: "sequential_7"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ dense_28 (Dense)                     │ (None, 128)                 │          10,496 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_29 (Dense)                     │ (None, 64)                  │           8,256 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 32)                  │           2,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 1)                   │              33 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 20,865 (81.50 KB)

 Trainable params: 20,865 (81.50 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - loss: 871.6337 - val_loss: 569.3903
Epoch 2/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 431.3995 - val_loss: 563.5383
Epoch 3/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 465.2407 - val_loss: 567.2416
Epoch 4/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 403.7716 - val_loss: 562.9581
Epoch 5/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 419.5273 - val_loss: 553.5831
Epoch 6/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 310.8563 - val_loss: 552.9531
Epoch 7/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 397.3064 - val_loss: 550.3994
Epoch 8/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - loss: 413.2908 - val_loss: 558.2197
Epoch 9/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 311.7236 - val_loss: 567.1912
Epoch 10/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 409.8815 - val_loss: 542.3641
Epoch 11/50
162/162 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - loss: 444.0722 - val_loss: 550.6978
Epoch 12

28.592288505793512

In [23]:
model_pred=model.predict(test_df)

sub = pd.DataFrame()
sub['id'] = test['id']
sub['pm2_5'] = model_pred
sub['pm2_5'].loc[sub['pm2_5'] <=1.2] = 1.2
sub.to_csv("mod.csv", index=False)
sub

87/87 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step


,id,pm2_5
0,id_ihxgrbq8bw,19.126123
1,id_dg6s4fhiwe,22.575518
2,id_f7hwwtmuzp,14.860468
3,id_ioese5awdg,23.646437
4,id_hdw320zpls,23.156534
...,...,...
2778,id_5yblexf7zp,12.215950
2779,id_c9cycuvetl,37.441666
2780,id_8xu1ardxni,62.469078
2781,id_aumy97t1iu,58.102467


In [24]:
sub = pd.DataFrame()
sub['y_test'] = y_test
sub['predictions'] = y_pred
sub['target>pred error'] = (sub.y_test - sub.predictions)
sub['pred>target error'] = (sub.predictions - sub.y_test)
sub.sort_values(by = 'target>pred error', ascending = False, inplace = True)
sub

,y_test,predictions,target>pred error,pred>target error
5400,428.266000,18.966896,409.299104,-409.299104
4454,403.490000,25.308222,378.181778,-378.181778
4438,377.314375,18.843817,358.470558,-358.470558
4434,330.450000,22.580355,307.869645,-307.869645
4442,217.304107,23.168468,194.135639,-194.135639
...,...,...,...,...
7686,69.360577,254.058945,-184.698368,184.698368
5589,66.253235,254.050293,-187.797058,187.797058
1055,23.887500,241.099823,-217.212323,217.212323
2080,4.795769,227.216751,-222.420982,222.420982


In [ ]:
def create_model():
    model = Sequential()
    model.add(Dense(128, input_dim=81, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dropout(0.5))
    model.add(Dense(32, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)))
    model.add(Dense(1))
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), loss='mean_squared_error')
    return model

k = 5  # Number of folds
kf = KFold(n_splits=k, shuffle=True, random_state=42)

mse_scores = []

for train_index, test_index in kf.split(X):
    X_train, X_val = X[train_index], X[test_index]
    y_train, y_val = y[train_index], y[test_index]
    
    # Create a new model instance
    model = create_model()
    
    # Train the model
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val), verbose=0)
    
    # Predict on validation data
    y_pred = model.predict(X_val)
    
    # Calculate the mean squared error
    mse = mean_squared_error(y_val, y_pred)
    mse_scores.append(mse)

# Calculate the average MSE across all folds
average_mse = np.mean(mse_scores)
print(f'Average MSE from {k}-fold cross-validation: {average_mse}')

final_model = create_model()
final_model.fit(X, y, epochs=50, batch_size=32, verbose=0)
predictions = final_model.predict(X)
print(mean_squared_error(y, predictions, squared=False))#7.3

In [ ]:
mean_squared_error(y_test, y_pred, squared=False)#7.3